In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

import matplotlib.pyplot as plt
from matplotlib import colors
import itertools

In [2]:
# @qml.op_transform
# def trace(op):
#     try:
#         return qml.math.real(qml.math.sum(op.eigvals()))
#     except qml.operation.EigvalsUndefinedError:
#         return qml.math.real(qml.math.trace(op.matrix()))

In [3]:


dev = qml.device("default.qubit", wires=4)

def levi_civita(i,j,k):
    return (i-j)*(j-k)*(k-i)/2


@qml.qnode(dev, interface = 'autograd')
def qcircuit0(x,param, index):
    p0 = [0.1,0.3,0.2]

    qml.Rot(*x[0], wires=0)
    qml.Rot(*x[1], wires=1)
    qml.Rot(*x[2], wires=2)
    qml.Rot(*x[3], wires=3)
    
#     qml.Rot(*p0, wires=0)
#     qml.Rot(*p0, wires=1)
#     qml.Rot(*p0, wires=2)
#     qml.Rot(*p0, wires=3)

    #equivariant ops
    qml.exp(qml.SWAP(wires=[0,1]), coeff=-0.5j*param[0])
    qml.exp(qml.SWAP(wires=[2,3]), coeff=-0.5j*param[1])
    qml.exp(qml.SWAP(wires=[1,2]), coeff=-0.5j*param[2])
    qml.exp(qml.SWAP(wires=[3,0]), coeff=-0.5j*param[3])
    
#     print(qml.density_matrix([0,1]))
    if index==0:
        return qml.density_matrix([0,1])
    elif index==1:
        return qml.density_matrix([2,3])
    elif len(index)==2:
        return qml.density_matrix([index[1]])

dev2 = qml.device('default.mixed', wires=4)
@qml.qnode(dev2, interface = 'autograd')
def qcircuit1(rho,y, param):
    qml.QubitDensityMatrix(rho, wires=[0,1])
    
#     p0 = [0.1,0.3,0.2]
#     qml.Rot(*p0, wires=0)
#     qml.Rot(*p0, wires=1)
#     qml.exp(qml.SWAP(wires=[0,1]), coeff=-0.5j*param[0])
    
    return qml.expval(qml.Hermitian(y, wires=[0,1]))

def compute_phis(rho , rhos, P):
    phi1 = qml.matrix(np.trace(rho)*qml.Identity(wires = 0))/2
    phi2 = rhos[0]-phi1
    phi3 = rhos[1]-phi1
    Pauli = [qml.matrix(qml.PauliX(wires = 0)+qml.PauliX(wires = 1)),qml.matrix(qml.PauliY(wires = 0)+qml.PauliY(wires = 1)),qml.matrix(qml.PauliZ(wires = 0)+qml.PauliZ(wires = 1))]
    pauli =[qml.matrix(qml.PauliX(wires = 0)),qml.matrix(qml.PauliY(wires = 0)),qml.matrix(qml.PauliZ(wires = 0))] 
    phi4 = np.zeros((2,2),dtype='complex128')
    for i,j,k in itertools.permutations([0,1,2]):
        phi4+= np.trace(rho*Pauli[i]*Pauli[j])*levi_civita(i,j,k)*pauli[k]
        
    return phi1+P[0]*phi2+P[1]*phi3+P[2]*phi4

In [4]:
def full_circuit(X, Params, VectorParams):
    
    Observ = qml.matrix(qml.SWAP(wires=[0,1]))
    densities = np.zeros((2,4,4),dtype='complex128')
    densities_single = np.zeros((4,2,2),dtype='complex128')
    for i in range(2):
        densities[i] = qcircuit0(x = X, param = Params,index=i)
    for i in range(4):
        densities_single[i] = qcircuit0(x = X, param = Params,index=[2,i])
    phi0 = compute_phis(densities[0],densities_single[0:2],VectorParams[0])
    phi1 = compute_phis(densities[1],densities_single[2:4],VectorParams[1])
    Rho = np.kron(phi0,phi1)
#     print(Rho)
#    print(qml.math.purity(Rho, [0, 1]))
    out = qcircuit1(Rho,y=Observ,param = [Params[-1]])
    
    return out

In [5]:
Params = np.random.rand(5, requires_grad=False)
VectorParams = np.random.rand(2,3, requires_grad=False)
X = np.random.rand(4, 3, requires_grad=False)

K = full_circuit(X, Params, VectorParams)

In [6]:
print(K)

0.7823179477205525


If we add an overall SU(2) rotation at the input stage, the result does not change

In [7]:
@qml.qnode(dev, interface = 'autograd')
def qcircuit0(x,param, index):
    p0 = [0.1,0.3,0.2]

    qml.Rot(*x[0], wires=0)
    qml.Rot(*x[1], wires=1)
    qml.Rot(*x[2], wires=2)
    qml.Rot(*x[3], wires=3)
    
    qml.Rot(*p0, wires=0)
    qml.Rot(*p0, wires=1)
    qml.Rot(*p0, wires=2)
    qml.Rot(*p0, wires=3)

    #equivariant ops
    qml.exp(qml.SWAP(wires=[0,1]), coeff=-0.5j*param[0])
    qml.exp(qml.SWAP(wires=[2,3]), coeff=-0.5j*param[1])
    qml.exp(qml.SWAP(wires=[1,2]), coeff=-0.5j*param[2])
    qml.exp(qml.SWAP(wires=[3,0]), coeff=-0.5j*param[3])
    
#     print(qml.density_matrix([0,1]))
    if index==0:
        return qml.density_matrix([0,1])
    elif index==1:
        return qml.density_matrix([2,3])
    elif len(index)==2:
        return qml.density_matrix([index[1]])
    
K = full_circuit(X, Params, VectorParams)
print(K)

0.7823179477205499


In fact, this Symmetry can occur at any stage of the circuit and does not change the result, hence shows that all the operators are indeed equivariant.

In [8]:
@qml.qnode(dev2, interface = 'autograd')
def qcircuit1(rho,y, param):
    qml.QubitDensityMatrix(rho, wires=[0,1])
    
#     p0 = [0.1,0.3,0.2]
#     qml.Rot(*p0, wires=0)
#     qml.Rot(*p0, wires=1)
#     qml.exp(qml.SWAP(wires=[0,1]), coeff=-0.5j*param[0])
    
    return qml.expval(qml.Hermitian(y, wires=[0,1]))

K = full_circuit(X, Params, VectorParams)
print(K)

0.7823179477205499
